In [ ]:
# База
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Нейронные сети
import tensorflow as tf
from tensorflow.keras.models import Sequential
# Свертка
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# Полносвязная
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Визуализация
import plotly.express as px
from matplotlib.image import imread

In [ ]:
# Важные константы
# Для воспроизводимости
SEED = 202212
# Размер изображений
SHAPE = (28, 28)
# Сохраненная модель
SAVE_PATH = 'weights/maximdu_cnn'

In [ ]:
tf.random.set_seed(SEED)

## Загружаем данные

> **Источник данных:**
>
> https://www.kaggle.com/datasets/oddrationale/mnist-in-csv

*Так как разведочный анализ уже был в прошлой работе, пропускаем*

In [ ]:
# Датасет уже разделен на две части,
# но мы потом поделим сами
df = pd.concat(
    [
        pd.read_csv('data/mnist_train.csv'),
        pd.read_csv('data/mnist_test.csv'),
    ], 
    ignore_index=True
)

In [ ]:
# Отделяем целевую переменную
X = df.drop(columns=['label'])
X = X.values

y = df['label']
y = y.values

## Преобразования

In [ ]:
# Преобразуем для обучения
# Перевод к [0,1]
X = X / 255
# Для перевода к тензору
X = X.reshape((-1, 28, 28, 1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=SEED,
    stratify=y
)

In [ ]:
# Переводим в тензор
# float16 для экономии памяти
X_train = tf.constant(X_train, dtype='float16')
X_test = tf.constant(X_test, dtype='float16')

In [ ]:
# Переводим целевую переменную в One Hot
y_train_onehot = tf.keras.utils.to_categorical(y_train)
y_test_onehot = tf.keras.utils.to_categorical(y_test)

## Построение модели

In [ ]:
def create_model():
    
    model = Sequential()

    model.add(Conv2D(filters=32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64,kernel_size=5,activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(0.25))

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
model = create_model()

In [ ]:
# Обучение модели
# Если обучается долго, можно загрузить веса
# в ячейке с model.load_weights

history = model.fit(
    X_train, 
    y_train_onehot, 
    epochs=8, 
    validation_data=(X_test, y_test_onehot)
)

In [ ]:
df_metrics = pd.DataFrame()
df_metrics['Точность на обучении (%)'] = history.history['accuracy']
df_metrics['Точность на тесте (%)'] = history.history['val_accuracy']
df_metrics = df_metrics * 100
df_metrics['Эпоха'] = df_metrics.index + 1

fig = px.line(
    df_metrics,
    x='Эпоха',
    y=['Точность на обучении (%)', 'Точность на тесте (%)'],
    template='plotly_white'
)

fig.update_traces(
    mode='lines+markers'
)

fig.data[0].line.color = 'black'
fig.data[1].line.color = 'green'

fig.update_layout(
    legend_yanchor='bottom',
    legend_xanchor='right',
    legend_y=0.1,
    legend_x=0.9,
    legend_title_text='',
    
    height=450,
    width=980
)

fig.show()

In [ ]:
model.save_weights(SAVE_PATH);

## Результаты обучения

In [ ]:
model = create_model()
model.load_weights(SAVE_PATH);

In [ ]:
# Получаем предсказания
pred_proba = model.predict(X_test)
pred_label = pred_proba.argmax(axis=1)

### Посмотрим неправильно предсказанные

In [ ]:
wrong = ( pred_label != y_test )

In [ ]:
def show_image(matrix):
    """Показывает изображение"""
    
    fig = px.imshow(
        matrix, 
        color_continuous_scale=['white', 'black']
    )
    fig.update_layout(
        width=650, 
        height=650, 
        coloraxis_showscale=False
    )
    
    fig.show()

In [ ]:
for k in range(10):
    print('Пометка из датасета:', y_test[wrong][k] )
    print('Предсказанное значение:', pred_label[wrong][k] )
    print('Предсказанные вероятности:', pred_proba[wrong][k].round(2) )
    show_image( X_test[wrong][k][:, :, 0] )

### Загрузим свое изображение в сеть

In [ ]:
image_path = 'data/my5.png'
image = imread(image_path)
image = 1 - image[:, :, 0]

In [ ]:
show_image(image)

In [ ]:
model.predict(
    (image).reshape(-1, 28, 28, 1)
).argmax(axis=1)

### Посмотрим фильтры свертки

In [ ]:
for k_filter in range(5):

    filter_ = np.array(
        model.layers[0].kernel[:, :, 0, k_filter]
    )

    fig = px.imshow(
        filter_, 
        color_continuous_scale=['red', 'white', 'blue'],
        color_continuous_midpoint=0,
    )
    fig.update_layout(
        width=650, 
        height=650, 
        coloraxis_showscale=True
    )

    fig.show()